In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import requests
import base64
import access

# Função para obter o token de acesso do Spotify
def get_spotify_access_token(client_id, client_secret):
    token_url = 'https://accounts.spotify.com/api/token'
    token_data = {'grant_type': 'client_credentials'}
    credentials = base64.b64encode(f'{client_id}:{client_secret}'.encode('utf-8')).decode('utf-8')
    headers = {'Authorization': f'Basic {credentials}'}
    response = requests.post(token_url, headers=headers, data=token_data)

    if response.status_code == 200:
        token = response.json().get('access_token')
        return token
    else:
        print(f'Erro ao obter token. Código de status: {response.status_code}')
        print('Resposta da API:')
        print(response.text)
        return None

# Função para buscar podcasts no Spotify
def search_spotify_podcasts(query, token):
    search_url = 'https://api.spotify.com/v1/search'
    params = {'query': query, 'type': 'show', 'market': 'BR', 'limit': 50}
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(search_url, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Erro na busca do Spotify. Código de status: {response.status_code}')
        print('Resposta da API:')
        print(response.text)
        return None

# Função para carregar dados no BigQuery
def load_to_bigquery(df, dataset_id, table_id, credentials):
    try:
        client = bigquery.Client(credentials=credentials)
        table_ref = client.dataset(dataset_id).table(table_id)

        job_config = bigquery.LoadJobConfig(
            skip_leading_rows=0,
            source_format=bigquery.SourceFormat.CSV,
        )

        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()  # Aguardar término do job

        print(f'Dados carregados na tabela {table_id} no BigQuery.')
    except Exception as e:
        print(f'Erro ao carregar dados no BigQuery: {e}')


# Definir as credenciais do Spotify
client_id = access.client_id
client_secret = access.secret_id 

# Obter o token de acesso do Spotify
spotify_token = get_spotify_access_token(client_id, client_secret)

if spotify_token:
    # Buscar podcasts no Spotify
    query = 'data hackers'
    resultados_api = search_spotify_podcasts(query, spotify_token)

    if resultados_api:
        # Extrair a lista de itens dos resultados
        items = resultados_api['shows'].get('items', [])

        # Criar uma única instância de DataFrame
        df_resultados = pd.DataFrame(columns=['name', 'description', 'id', 'total_episodes'])
        # Criar uma lista de dicionários
        list_resultados = []
        for item in items:
            if item and all(field in item for field in ['name', 'description', 'id', 'total_episodes']):
                new_row = {
                    'name': item.get('name'),
                    'description': item.get('description'),
                    'id': item.get('id'),
                    'total_episodes': item.get('total_episodes'),
                }
                list_resultados.append(new_row)



        # Caminho para a chave do IAM do BigQuery
        key_path = "C:/Users/acoli/OneDrive/Documentos/Projetos_Python/projeto_gb/access/GB.json"

        # Carregar credenciais do BigQuery
        bigquery_credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

        # Substituir com as informações do seu projeto, dataset e tabela no BigQuery
        dataset_id = 'transient'
        table_id = 'stg_podcasts_pesquisa'
print(list_resultados)
# Carregar dados no BigQuery
if list_resultados:
    print('Dataframe com todos os resultados:')
    df_resultados = pd.DataFrame(list_resultados)
    print(df_resultados)
    load_to_bigquery(df_resultados, dataset_id, table_id, bigquery_credentials)
else:
    print('DataFrame vazio. Nenhum dado será carregado no BigQuery.')



TypeError: argument of type 'NoneType' is not iterable